# Build a cuisine recommender

In [1]:
# Install skl2onnx
! pip install skl2onnx

  Obtaining dependency information for skl2onnx from https://files.pythonhosted.org/packages/c0/d1/ef96f715f14ab4a11a4382e3eb9fc7a57ee59e3527253a7b21d62ca20402/skl2onnx-1.15.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for onnx>=1.2.1 from https://files.pythonhosted.org/packages/03/49/7263b3806ffebd3c967341986df32a5e62b2fa78beca2cdf9516d876b3fc/onnx-1.14.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for onnxconverter-common>=1.7.0 from https://files.pythonhosted.org/packages/6d/6a/9ed9fd4da34cb41fda35bc5cc9e990c605689db7de63ed84fedbca5a77f6/onnxconverter_common-1.14.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for protobuf>=3.20.2 from https://files.pythonhosted.org/packages/c2/59/f89c04923d68595d359f4cd7adbbdf5e5d791257945f8873d88b2fd1f979/protobuf-4.24.4-cp310-abi3-win_amd64.whl.metadata
     ---------------------------------------- 0.0/162.1 kB ? eta -:--:--
     ----------------- --------------------- 71.7/162.1 

In [2]:
# Import relevant library
import pandas as pd

In [3]:
# Read the cleaned cuisine data
data = pd.read_csv('../data/cleaned_cuisines.csv')
data.head()

,Unnamed: 0,cuisine,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,...,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,0,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,indian,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [4]:
# Remove the first two unnecessary columns and store it as the features
X = data.iloc[:,2:]
X.head()

,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,artemisia,artichoke,...,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [5]:
# Create the labels
y = data[['cuisine']]
y.head()

,cuisine
0,indian
1,indian
2,indian
3,indian
4,indian


## Building the model

In [6]:
# Import the appropriate libraries
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix,classification_report

In [7]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [8]:
# Create an SVC model and fit the dataset to it
model = SVC(kernel='linear', C=10, probability=True,random_state=0)
model.fit(X_train,y_train.values.ravel())

SVC(C=10, kernel='linear', probability=True, random_state=0)

In [9]:
# Test the model
y_pred = model.predict(X_test)

In [10]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

     chinese       0.68      0.69      0.68       221
      indian       0.89      0.87      0.88       249
    japanese       0.81      0.75      0.78       255
      korean       0.82      0.74      0.78       231
        thai       0.74      0.86      0.79       243

    accuracy                           0.78      1199
   macro avg       0.79      0.78      0.78      1199
weighted avg       0.79      0.78      0.79      1199



## Convert the model to Onnx

In [11]:
# Import the relevant libraries
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('float_input', FloatTensorType([None, 380]))]
options = {id(model): {'nocl': True, 'zipmap': False}}

In [12]:
# Convert the sklearn to onnx
onx = convert_sklearn(model, initial_types=initial_type, options=options)

# Save the converted model as a file
with open("./model.onnx", "wb") as f:
    f.write(onx.SerializeToString())